# In this code we will use the downloded images to genrate a image dataset for further data processing.


## download images folder from AWS

In [1]:
!pip install boto3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.1 MB/s eta 0:00:00


## loading credentails from Collab env

In [2]:
#Access your secret keys in Python via:
from google.colab import userdata

BUCKET_NAME =userdata.get('cvBucket')
ACCESS_KEY= userdata.get('accessKey')
SECRET_KEY= userdata.get('secretKey')

#The object to interact with AWS S3

In [3]:
import boto3
# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key= SECRET_KEY)

#s3 is object created which is  authrosied using access keys & secret key to  access the S3 bucket in AWS

## Looking out for all avialable files within that bucket


In [4]:
# List the files in the specified S3 bucket
def list_files_in_bucket(bucket_name):
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.all():
        print(obj.key)

# Call the function to list files
list_files_in_bucket(BUCKET_NAME)

Test_Raw_images.zip
dataset.pkl
face_detector.h5
raw_image_files.zip
test_dataset.pkl


## here we get the name of file we are looking for--  "raw_image_files.zip"

In [5]:
import botocore
# save the filename in KEY variable
KEY = 'raw_image_files.zip' # replace with your object key

try:
  # we are trying to download file from s3 with name `raw_image_files.zip` stored in varaible KEY
  # to colab dir with name `raw_image_files.zip`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'raw_image_files.zip')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

##Exctracting the Zip File

In [6]:
import zipfile
import os

# Create the extraction directory-dataset
if not os.path.exists("/content/dataset"):
  os.makedirs("/content/dataset")


# Extract the ZIP file
with zipfile.ZipFile("/content/raw_image_files.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


## Let python walk through the directory and read out files from nested folder and list them out.

In [7]:
def list_files_in_directory(directory):
    for root, dirs, files in os.walk(directory):
        level = root.replace(directory, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f"{indent}{os.path.basename(root)}/")
        sub_indent = ' ' * 4 * (level + 1)
        for f in files:
            print(f"{sub_indent}{f}")

# List the extracted files
list_files_in_directory("/content/dataset")


dataset/
    Raw Images/
        alarming/
            human face with multiple coverings/
                Image_57.jpg
                Image_14.jpg
                Image_21.jpg
                Image_59.jpg
                Image_24.jpg
                Image_98.jpg
                Image_12.jpg
                Image_68.jpg
                Image_97.jpg
                Image_38.jpg
                Image_32.jpg
                Image_45.jpeg
                Image_29.jpeg
                Image_34.jpg
                Image_58.jpg
                Image_30.jpg
                Image_3.jpg
                Image_51.jpg
                Image_15.jpg
                Image_88.jpg
                Image_89.jpg
                Image_70.jpg
                Image_10.jpg
                Image_2.jpg
                Image_1.jpg
                Image_49.jpg
                Image_27.jpg
                Image_31.jpg
                Image_33.jpg
                Image_23.jpg
                Image_87.jpg
           

## Now we know  a images are inside the nested folder. We will use OpenCV to read these images and create a single dataset from it.

In [8]:
!pip install opencv-Python pandas

In [10]:
import cv2
import pandas as pd

# Define the root directory of your dataset
root_dir = '/content/dataset'

# Initialize a list to store image data and labels
data = []

# Define the function to read images from a directory
def read_images_from_directory(directory, label):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):  # Add other image file extensions if necessary
                file_path = os.path.join(root, file)
                # Read the image using OpenCV
                image = cv2.imread(file_path)
                if image is not None:
                  resized_image = cv2.resize(image, (224,224))
                  normalized_image = resized_image / 255.0

                  # Append the image data and label to the list
                  data.append((file_path,normalized_image, label))

# Read images from the "acceptable" directory
acceptable_dir = os.path.join(root_dir, 'acceptable')
read_images_from_directory(acceptable_dir, 'acceptable')

# Read images from the "alarming" directory
alarming_dir = os.path.join(root_dir, 'alarming')
read_images_from_directory(alarming_dir, 'alarming')

# Convert the list to a DataFrame
df = pd.DataFrame(data, columns=['file_path', 'normalized_image', 'label'])

# Display the first few rows of the DataFrame
print(df.head())

                                           file_path  \
0  /content/dataset/acceptable/human face without...   
1  /content/dataset/acceptable/human face without...   
2  /content/dataset/acceptable/human face without...   
3  /content/dataset/acceptable/human face without...   
4  /content/dataset/acceptable/human face without...   

                                    normalized_image       label  
0  [[[0.2901960784313726, 0.3215686274509804, 0.3...  acceptable  
1  [[[0.9882352941176471, 0.984313725490196, 0.98...  acceptable  
2  [[[0.8235294117647058, 0.8156862745098039, 0.8...  acceptable  
3  [[[0.8313725490196079, 0.8823529411764706, 0.9...  acceptable  
4  [[[0.49411764705882355, 0.21176470588235294, 0...  acceptable  


In [11]:
df.head()

,file_path,normalized_image,label
0,/content/dataset/acceptable/human face without...,"[[[0.2901960784313726, 0.3215686274509804, 0.3...",acceptable
1,/content/dataset/acceptable/human face without...,"[[[0.9882352941176471, 0.984313725490196, 0.98...",acceptable
2,/content/dataset/acceptable/human face without...,"[[[0.8235294117647058, 0.8156862745098039, 0.8...",acceptable
3,/content/dataset/acceptable/human face without...,"[[[0.8313725490196079, 0.8823529411764706, 0.9...",acceptable
4,/content/dataset/acceptable/human face without...,"[[[0.49411764705882355, 0.21176470588235294, 0...",acceptable


In [12]:
df['normalized_image'][0].shape

(224, 224, 3)

In [13]:
type(df['normalized_image'][0])

numpy.ndarray

In [14]:
import random
import matplotlib.pyplot as plt

# Function to visualize randomly selected images from the DataFrame
def visualize_random_images(df, num_images):
    # Randomly select indices
    random_indices = random.sample(range(len(df)), num_images)

    # Iterate over randomly selected indices
    for index in random_indices:
        # Get the row at the selected index
        row = df.iloc[index]
        # Reconstruct the image from normalized pixel values
        image = (row['normalized_image'] * 255).astype('uint8')
        # Convert the image from BGR to RGB (OpenCV uses BGR format)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Display the image
        plt.imshow(image)
        plt.title(row['label'])
        plt.axis('off')
        plt.show()

# Visualize 2 randomly selected images from the DataFrame
visualize_random_images(df, num_images=9)


Output hidden; open in https://colab.research.google.com to view.

## Now lets save this Dataset. But NOTE :SAVING This to csv will convert ndtype.arrays in column "Normalised Image" will be saved as plain text. and We would need to restore them back as ndtype array , before feeding to model.

## Hence instead of to_csv, we will pikle it to preserv all the datatype

In [15]:
import pickle

with open('dataset.pkl', 'wb') as f:
    pickle.dump(df, f)

In [16]:
#Creating S3 object that will  push this dataset
push_to_s3= boto3.client('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)



#Using object pushing/ uploading image datset to S3
push_to_s3.upload_file('/content/dataset.pkl',BUCKET_NAME, 'dataset.pkl')

## Creating Test dataset


In [ ]:
import botocore

# save the filename in KEY variable
KEY = 'Test_Raw_images.zip' # replace with your object key

try:
  # we are trying to download file from s3 with name `raw_image_files.zip` stored in varaible KEY
  # to colab dir with name `raw_image_files.zip`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'Test_Raw_images.zip')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

In [ ]:
import zipfile
import os

# Extract the ZIP file
with zipfile.ZipFile("/content/Test_Raw_images.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
import cv2
import pandas as pd

# Define the root directory of your dataset
root_dir = '/content/Test_Raw_images'

# Initialize a list to store image data and labels
test_data = []

# Define the function to read images from a directory
def read_images_from_test_directory(directory, label):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):  # Add other image file extensions if necessary
                file_path = os.path.join(root, file)
                # Read the image using OpenCV
                image = cv2.imread(file_path)
                if image is not None:
                  resized_image = cv2.resize(image, (224,224))
                  normalized_image = resized_image / 255.0

                  # Append the image data and label to the list
                  test_data.append((file_path,normalized_image, label))

# Read images from the "acceptable" directory
acceptable_dir = os.path.join(root_dir, 'acceptable')
read_images_from_test_directory(acceptable_dir, 'acceptable')

# Read images from the "alarming" directory
alarming_dir = os.path.join(root_dir, 'alarming')
read_images_from_test_directory(alarming_dir, 'alarming')

# Convert the list to a DataFrame
test_df = pd.DataFrame(test_data, columns=['file_path', 'normalized_image', 'label'])

# Display the first few rows of the DataFrame
test_df.head()

,file_path,normalized_image,label
0,/content/Test_Raw_images/acceptable/images (9)...,"[[[0.6901960784313725, 0.7333333333333333, 0.7...",acceptable
1,/content/Test_Raw_images/acceptable/download (...,"[[[0.6274509803921569, 0.7372549019607844, 0.8...",acceptable
2,/content/Test_Raw_images/acceptable/images (6)...,"[[[0.08627450980392157, 0.06666666666666667, 0...",acceptable
3,/content/Test_Raw_images/acceptable/download (...,"[[[0.47843137254901963, 0.49019607843137253, 0...",acceptable
4,/content/Test_Raw_images/acceptable/images (8)...,"[[[0.1607843137254902, 0.3803921568627451, 0.5...",acceptable


In [ ]:
type(test_df['normalized_image'][0]),test_df['normalized_image'][0].shape

(numpy.ndarray, (224, 224, 3))

In [ ]:
test_df['label'].value_counts()

label
alarming      28
acceptable    27
Name: count, dtype: int64

In [ ]:
import pickle

with open('test_dataset.pkl', 'wb') as f:
    pickle.dump(test_df, f)

In [ ]:
#Creating S3 object that will  push this dataset
push_to_s3= boto3.client('s3',aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY)



#Using object pushing/ uploading image datset to S3
push_to_s3.upload_file('/content/test_dataset.pkl',BUCKET_NAME, 'test_dataset.pkl')